# Handwriting Recognition: <br> World Vision Project

### Camilo Salazar <br> October 7th 2024

In [6]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [10]:
train = pd.read_csv('data/written_name_train_v2.csv')